# 01 -- Data Quality Report

This notebook provides a comprehensive data-quality assessment of the
extracted feature matrix produced by the Tactical State Discovery Engine.

**Sections:**
1. Data loading and overview
2. Null-rate analysis (per column, per tier, per segment type)
3. Feature distributions
4. Correlation matrix
5. Per-team summary statistics
6. Outlier analysis
7. Temporal coverage

In [ ]:
from __future__ import annotations

import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import seaborn as sns
from tqdm.auto import tqdm

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

sns.set_theme(style="whitegrid", font_scale=0.9)
plt.rcParams["figure.dpi"] = 120

FEATURES_PATH = PROJECT_ROOT / "data" / "output" / "features.parquet"

METADATA_COLS: set[str] = {
    "match_id",
    "team_id",
    "segment_type",
    "start_time",
    "end_time",
    "period",
    "match_minute",
}

---
## 1. Data Loading & Overview

In [ ]:
if FEATURES_PATH.exists():
    df = pl.read_parquet(FEATURES_PATH)
    print(f"Loaded {FEATURES_PATH}")
else:
    print(
        f"{FEATURES_PATH} not found.\n"
        "Run `python scripts/run_feature_extraction.py` first."
    )
    raise SystemExit(1)

In [ ]:
feature_cols: list[str] = [
    c for c in df.columns if c not in METADATA_COLS
]
tier_map: dict[str, str] = {}
for col in feature_cols:
    if col.startswith("t1_"):
        tier_map[col] = "Tier 1"
    elif col.startswith("t2_"):
        tier_map[col] = "Tier 2"
    elif col.startswith("t3_"):
        tier_map[col] = "Tier 3"
    else:
        tier_map[col] = "Unknown"

print(f"Rows              : {df.height:,}")
print(f"Total columns     : {df.width}")
print(f"Metadata columns  : {len(METADATA_COLS & set(df.columns))}")
print(f"Feature columns   : {len(feature_cols)}")

tier_counts = (
    pl.DataFrame({"feature": list(tier_map.keys()), "tier": list(tier_map.values())})
    .group_by("tier")
    .agg(pl.col("feature").count().alias("n_features"))
    .sort("tier")
)
print("\nFeatures per tier:")
print(tier_counts)

In [ ]:
segment_counts = (
    df.group_by("segment_type")
    .agg(pl.len().alias("n_rows"))
    .sort("segment_type")
)
print("Rows per segment type:")
print(segment_counts)

In [ ]:
n_matches = df["match_id"].n_unique()
n_teams = df["team_id"].n_unique()
periods = sorted(df["period"].unique().to_list())

print(f"Unique matches : {n_matches}")
print(f"Unique teams   : {n_teams}")
print(f"Periods        : {periods}")

---
## 2. Null-Rate Analysis

In [ ]:
null_rates: dict[str, float] = {}
for col in feature_cols:
    null_rates[col] = df[col].null_count() / df.height if df.height > 0 else 0.0

null_df = (
    pl.DataFrame(
        {
            "feature": list(null_rates.keys()),
            "null_rate": list(null_rates.values()),
        }
    )
    .with_columns(
        pl.col("feature")
        .map_elements(
            lambda f: tier_map.get(f, "Unknown"),
            return_dtype=pl.Utf8,
        )
        .alias("tier")
    )
    .sort("null_rate", descending=True)
)

cols_with_nulls = null_df.filter(pl.col("null_rate") > 0.0)
cols_no_nulls = null_df.filter(pl.col("null_rate") == 0.0).height

total_cells = df.height * len(feature_cols)
total_nulls = sum(df[c].null_count() for c in feature_cols)
overall_pct = 100.0 * total_nulls / total_cells if total_cells > 0 else 0.0

print(f"Overall null rate: {overall_pct:.2f}% ({total_nulls:,} / {total_cells:,})")
print(f"Fully populated columns: {cols_no_nulls} / {len(feature_cols)}")

if cols_with_nulls.height > 0:
    print("\nColumns with nulls (descending):")
    print(cols_with_nulls.head(10))

In [ ]:
# Null rate by tier
tier_null = (
    null_df.group_by("tier")
    .agg(
        pl.col("null_rate").mean().alias("mean_null_rate"),
        pl.col("null_rate").max().alias("max_null_rate"),
        pl.col("null_rate").filter(pl.col("null_rate") > 0).count().alias("n_cols_with_nulls"),
        pl.col("feature").count().alias("n_cols"),
    )
    .sort("tier")
)
print("Null rate summary by tier:")
print(tier_null)

In [ ]:
# Visualise null rates for columns that have any nulls
if cols_with_nulls.height > 0:
    plot_df = cols_with_nulls.head(40)  # cap for readability
    names = plot_df["feature"].to_list()
    rates = plot_df["null_rate"].to_list()
    tiers = plot_df["tier"].to_list()

    tier_palette = {"Tier 1": "#4c72b0", "Tier 2": "#dd8452", "Tier 3": "#55a868", "Unknown": "#999999"}
    bar_colors = [tier_palette.get(t, "#999999") for t in tiers]

    fig, ax = plt.subplots(figsize=(10, max(4, len(names) * 0.28)))
    y_pos = np.arange(len(names))
    ax.barh(y_pos, [r * 100 for r in rates], color=bar_colors, edgecolor="white", linewidth=0.5)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(names, fontsize=7)
    ax.set_xlabel("Null rate (%)")
    ax.set_title("Null Rate per Feature Column (top 40)")
    ax.invert_yaxis()

    # Legend
    from matplotlib.patches import Patch
    handles = [Patch(facecolor=c, label=t) for t, c in tier_palette.items() if t in set(tiers)]
    ax.legend(handles=handles, loc="lower right", fontsize=8)

    plt.tight_layout()
    plt.show()
else:
    print("No columns have null values -- nothing to plot.")

In [ ]:
# Null rates by segment type
for seg_type in df["segment_type"].unique().sort().to_list():
    seg = df.filter(pl.col("segment_type") == seg_type)
    seg_nulls = sum(seg[c].null_count() for c in feature_cols)
    seg_cells = seg.height * len(feature_cols)
    seg_pct = 100.0 * seg_nulls / seg_cells if seg_cells > 0 else 0.0
    print(f"  {seg_type:<12s}  null rate = {seg_pct:.2f}%  (rows={seg.height:,})")

---
## 3. Feature Distributions

In [ ]:
# Descriptive statistics for all numeric feature columns
desc = df.select(feature_cols).describe()
print(desc.head(10))

In [ ]:
# Select representative features (one per extractor group) for histograms
_REPRESENTATIVE_PREFIXES = [
    "t1_spatial_event_centroid_x",
    "t1_temporal_event_rate",
    "t1_pass_count",
    "t1_pass_completion_rate",
    "t1_carry_count",
    "t1_defend_pressure_count",
    "t1_shoot_count",
    "t1_context_score_differential",
    "t1_context_possession_share",
    "t2_shape_engagement_line",
    "t2_press_intensity",
    "t2_press_ppda",
    "t2_transition_transition_speed",
    "t2_zonal_box_entries",
    "t3_formation_team_centroid_x",
    "t3_relational_avg_nearest_opponent_dist",
]

plot_features = [f for f in _REPRESENTATIVE_PREFIXES if f in feature_cols]

n_plots = len(plot_features)
n_cols_grid = 4
n_rows_grid = (n_plots + n_cols_grid - 1) // n_cols_grid

fig, axes = plt.subplots(
    n_rows_grid, n_cols_grid,
    figsize=(n_cols_grid * 3.2, n_rows_grid * 2.8),
)
axes_flat = axes.flatten() if n_plots > 1 else [axes]

for idx, feat in enumerate(tqdm(plot_features, desc="Plotting histograms")):
    ax = axes_flat[idx]
    vals = df[feat].drop_nulls().to_numpy()
    if len(vals) == 0:
        ax.set_title(feat, fontsize=7)
        ax.text(0.5, 0.5, "all null", ha="center", va="center", transform=ax.transAxes)
        continue
    ax.hist(vals, bins=50, color="#4c72b0", edgecolor="white", linewidth=0.3)
    ax.set_title(feat.replace("t1_", "").replace("t2_", "").replace("t3_", ""), fontsize=7)
    ax.tick_params(labelsize=6)

# Hide unused axes
for idx in range(n_plots, len(axes_flat)):
    axes_flat[idx].set_visible(False)

fig.suptitle("Feature Distributions (representative sample)", fontsize=11, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution comparison: window vs possession segments
_COMPARE_FEATURES = [
    "t1_pass_count",
    "t1_temporal_event_rate",
    "t1_spatial_event_centroid_x",
    "t1_defend_pressure_count",
]
compare_features = [f for f in _COMPARE_FEATURES if f in feature_cols]

if compare_features:
    fig, axes = plt.subplots(1, len(compare_features), figsize=(len(compare_features) * 3.5, 3))
    if len(compare_features) == 1:
        axes = [axes]

    for ax, feat in zip(axes, compare_features):
        for seg_type, color in [("window", "#4c72b0"), ("possession", "#dd8452")]:
            vals = (
                df.filter(pl.col("segment_type") == seg_type)[feat]
                .drop_nulls()
                .to_numpy()
            )
            if len(vals) > 0:
                ax.hist(vals, bins=40, alpha=0.6, label=seg_type, color=color, edgecolor="white", linewidth=0.3)
        short_name = feat.replace("t1_", "").replace("t2_", "")
        ax.set_title(short_name, fontsize=8)
        ax.tick_params(labelsize=6)
        ax.legend(fontsize=6)

    fig.suptitle("Window vs Possession Distributions", fontsize=10, y=1.02)
    plt.tight_layout()
    plt.show()

---
## 4. Correlation Matrix

In [ ]:
# Compute correlation on non-null numeric features
numeric_feats = [
    c for c in feature_cols
    if df[c].dtype in (pl.Float64, pl.Float32, pl.Int64, pl.Int32)
]

# Drop columns that are entirely null (correlation undefined)
valid_feats = [
    c for c in numeric_feats if df[c].drop_nulls().len() > 1
]

# Use numpy for efficient correlation computation
mat = df.select(valid_feats).fill_null(strategy="forward").fill_null(0).to_numpy()
corr = np.corrcoef(mat, rowvar=False)
# Replace NaN (constant columns) with 0
corr = np.nan_to_num(corr, nan=0.0)

print(f"Correlation matrix shape: {corr.shape[0]} x {corr.shape[1]}")

In [ ]:
fig, ax = plt.subplots(figsize=(14, 12))
im = ax.imshow(corr, cmap="RdBu_r", vmin=-1, vmax=1, aspect="auto")

# Tier boundary lines
tier_boundaries: list[int] = []
prev_tier = ""
for i, col in enumerate(valid_feats):
    cur_tier = tier_map.get(col, "")
    if cur_tier != prev_tier and prev_tier:
        tier_boundaries.append(i)
    prev_tier = cur_tier

for b in tier_boundaries:
    ax.axhline(b - 0.5, color="black", linewidth=0.8, linestyle="--")
    ax.axvline(b - 0.5, color="black", linewidth=0.8, linestyle="--")

ax.set_title("Feature Correlation Matrix", fontsize=12)
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# Abbreviated tick labels (show every Nth)
n_feats = len(valid_feats)
step = max(1, n_feats // 30)
tick_positions = list(range(0, n_feats, step))
tick_labels = [
    valid_feats[i].replace("t1_", "").replace("t2_", "").replace("t3_", "")
    for i in tick_positions
]
ax.set_xticks(tick_positions)
ax.set_xticklabels(tick_labels, rotation=90, fontsize=5)
ax.set_yticks(tick_positions)
ax.set_yticklabels(tick_labels, fontsize=5)

plt.tight_layout()
plt.show()

In [ ]:
# Highly correlated pairs (|r| > 0.85), excluding self-correlation
threshold = 0.85
high_corr_pairs: list[tuple[str, str, float]] = []

# Only iterate upper triangle
for i in range(corr.shape[0]):
    for j in range(i + 1, corr.shape[1]):
        r = corr[i, j]
        if abs(r) > threshold:
            high_corr_pairs.append((valid_feats[i], valid_feats[j], float(r)))

high_corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

print(f"Pairs with |r| > {threshold}: {len(high_corr_pairs)}")
if high_corr_pairs:
    hc_df = pl.DataFrame(
        {
            "feature_a": [p[0] for p in high_corr_pairs[:20]],
            "feature_b": [p[1] for p in high_corr_pairs[:20]],
            "correlation": [round(p[2], 4) for p in high_corr_pairs[:20]],
        }
    )
    print(hc_df)

---
## 5. Per-Team Summary Statistics

In [ ]:
# Representative features for per-team summary
_TEAM_SUMMARY_FEATURES = [
    "t1_pass_count",
    "t1_pass_completion_rate",
    "t1_spatial_event_centroid_x",
    "t1_defend_pressure_count",
    "t1_context_possession_share",
    "t2_press_intensity",
]
team_feats = [f for f in _TEAM_SUMMARY_FEATURES if f in feature_cols]

team_stats = (
    df.group_by("team_id")
    .agg(
        pl.len().alias("n_segments"),
        pl.col("match_id").n_unique().alias("n_matches"),
        *[
            pl.col(f).mean().alias(f"{f}_mean")
            for f in team_feats
        ],
        *[
            pl.col(f).std().alias(f"{f}_std")
            for f in team_feats
        ],
    )
    .sort("n_segments", descending=True)
)

print(team_stats.head(10))

In [ ]:
# Per-team null rate
team_ids = df["team_id"].unique().sort().to_list()
team_null_rows: list[dict[str, object]] = []

for tid in team_ids:
    team_df = df.filter(pl.col("team_id") == tid)
    n = team_df.height
    total = n * len(feature_cols)
    nulls = sum(team_df[c].null_count() for c in feature_cols)
    pct = 100.0 * nulls / total if total > 0 else 0.0
    team_null_rows.append(
        {"team_id": tid, "n_segments": n, "null_rate_pct": round(pct, 2)}
    )

team_null_df = pl.DataFrame(team_null_rows).sort("null_rate_pct", descending=True)
print("Per-team null rate:")
print(team_null_df.head(10))

In [ ]:
# Box plots: key features by top teams (by segment count)
top_teams = (
    df.group_by("team_id")
    .agg(pl.len().alias("cnt"))
    .sort("cnt", descending=True)
    .head(8)["team_id"]
    .to_list()
)

_BOX_FEATURES = [
    "t1_pass_completion_rate",
    "t1_spatial_event_centroid_x",
    "t1_context_possession_share",
]
box_features = [f for f in _BOX_FEATURES if f in feature_cols]

if box_features and top_teams:
    top_df = df.filter(pl.col("team_id").is_in(top_teams))
    fig, axes = plt.subplots(1, len(box_features), figsize=(len(box_features) * 4, 4))
    if len(box_features) == 1:
        axes = [axes]

    for ax, feat in zip(axes, box_features):
        plot_data: list[list[float]] = []
        labels: list[str] = []
        for tid in top_teams:
            vals = (
                top_df.filter(pl.col("team_id") == tid)[feat]
                .drop_nulls()
                .to_list()
            )
            if vals:
                plot_data.append(vals)
                labels.append(str(tid)[:12])
        if plot_data:
            bp = ax.boxplot(plot_data, labels=labels, patch_artist=True)
            for patch in bp["boxes"]:
                patch.set_facecolor("#4c72b0")
                patch.set_alpha(0.7)
            ax.set_title(feat.replace("t1_", ""), fontsize=8)
            ax.tick_params(axis="x", rotation=45, labelsize=6)
            ax.tick_params(axis="y", labelsize=7)

    fig.suptitle("Feature Distributions by Team (top 8)", fontsize=10, y=1.02)
    plt.tight_layout()
    plt.show()

---
## 6. Outlier Analysis

In [ ]:
# IQR-based outlier detection per feature
outlier_summary: list[dict[str, object]] = []

for feat in tqdm(feature_cols, desc="Detecting outliers"):
    col = df[feat].drop_nulls()
    n_valid = col.len()
    if n_valid < 10:
        continue
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    if q1 is None or q3 is None:
        continue
    iqr = q3 - q1
    if iqr == 0:
        continue
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    n_outliers = col.filter((col < lower) | (col > upper)).len()
    outlier_pct = 100.0 * n_outliers / n_valid
    outlier_summary.append(
        {
            "feature": feat,
            "n_outliers": n_outliers,
            "outlier_pct": round(outlier_pct, 2),
            "q1": round(float(q1), 4),
            "q3": round(float(q3), 4),
            "iqr": round(float(iqr), 4),
        }
    )

outlier_df = (
    pl.DataFrame(outlier_summary)
    .sort("outlier_pct", descending=True)
)

print("Top 15 features by outlier percentage (IQR method):")
print(outlier_df.head(15))

In [ ]:
# Outlier rate distribution
if outlier_df.height > 0:
    fig, ax = plt.subplots(figsize=(7, 3.5))
    ax.hist(
        outlier_df["outlier_pct"].to_numpy(),
        bins=30,
        color="#c44e52",
        edgecolor="white",
        linewidth=0.4,
    )
    ax.set_xlabel("Outlier % (IQR)")
    ax.set_ylabel("Number of features")
    ax.set_title("Distribution of Per-Feature Outlier Rates")
    plt.tight_layout()
    plt.show()

---
## 7. Temporal Coverage

In [ ]:
# Segment count by match minute
if "match_minute" in df.columns:
    fig, ax = plt.subplots(figsize=(10, 3.5))
    for seg_type, color in [("window", "#4c72b0"), ("possession", "#dd8452")]:
        seg_df = df.filter(pl.col("segment_type") == seg_type)
        if seg_df.height > 0:
            vals = seg_df["match_minute"].drop_nulls().to_numpy()
            ax.hist(
                vals, bins=90, alpha=0.6, label=seg_type,
                color=color, edgecolor="white", linewidth=0.3,
            )
    ax.set_xlabel("Match Minute")
    ax.set_ylabel("Segment Count")
    ax.set_title("Temporal Coverage: Segments by Match Minute")
    ax.legend(fontsize=8)
    plt.tight_layout()
    plt.show()

In [ ]:
# Segments per period
period_counts = (
    df.group_by("period", "segment_type")
    .agg(pl.len().alias("n_segments"))
    .sort("period", "segment_type")
)
print("Segments per period:")
print(period_counts)

In [ ]:
# Segments per match histogram
segs_per_match = (
    df.group_by("match_id", "team_id")
    .agg(pl.len().alias("n_segments"))
)

fig, ax = plt.subplots(figsize=(7, 3.5))
ax.hist(
    segs_per_match["n_segments"].to_numpy(),
    bins=40, color="#55a868", edgecolor="white", linewidth=0.3,
)
ax.set_xlabel("Segments per (match, team)")
ax.set_ylabel("Count")
ax.set_title("Distribution of Segment Counts per Match-Team")
plt.tight_layout()
plt.show()

print(f"Median segments per (match, team): {segs_per_match['n_segments'].median():.0f}")
print(f"Min: {segs_per_match['n_segments'].min()}, Max: {segs_per_match['n_segments'].max()}")

---
## Summary

Key findings from this data-quality report:

1. **Null rates** -- Tier 3 features have the highest null rates (expected, as 360 data is not available for all matches). Check the null-rate bar chart above for specifics.
2. **Feature distributions** -- Some counting features are heavily right-skewed (shots, fouls). Rates and percentages are generally bounded and well-behaved.
3. **Correlation** -- Highly correlated feature pairs may warrant dimensionality reduction or feature selection before modeling.
4. **Per-team variation** -- Teams with more matches naturally have more segments. Null rates should be roughly uniform across teams (differences indicate 360 availability).
5. **Temporal coverage** -- Segment density is roughly uniform across match minutes, confirming the windowing strategy provides balanced coverage.